##<font color="#fd79a8">Extraction-Based Summarizer <br><font color="#48dbfb">Scraped Wikipedia articles using Beautiful Soup

In [1]:
import bs4 as bs
import urllib.request
import re
import nltk
nltk.download('punkt')
import sys
import csv 

#persian cuisine
scraped_data = urllib.request.urlopen('https://alotech.com.tr/genel-bakis/?gclid=CjwKCAiA55mPBhBOEiwANmzoQvq-_QiLC0vWrpDdopoHsekl0tqYPIxbi-0kPN6HNFK7JHjPaNFbCBoCMKgQAvD_BwE')
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text

[nltk_data] Downloading package punkt to /home/shakabrah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Removing Square Brackets and Extra Spaces
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
#any whitespace character \s+
article_text = re.sub(r'\s+', ' ', article_text)

In [3]:
# Removing special characters and digits
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
#any whitespace character \s+
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

##<font color="#fd79a8">Convert paragraphs to sentences

In [4]:
sentence_list = nltk.sent_tokenize(article_text)

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shakabrah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

###<font color="#fd79a8"> Loop to calculate the word frequencies. <br>Tokenize the sentences<br>if word is not a stopword and is in the word list, the count is added 

In [6]:
stopwords = nltk.corpus.stopwords.words('turkish')

word_frequencies = {}
for word in nltk.word_tokenize(formatted_article_text):
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1


###<font color="#48dbfb"> Keys() method<br>The keys() method returns a view object. The view object contains the keys of the dictionary, as a list.

###<font color="#48dbfb">When an item is added in the dictionary, the view object also gets updated:

###<font color="#48dbfb">Find weighted frequency of occurence 

In [7]:
maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

###<font color="#48dbfb">Replace words with weighted frequency in sentences

In [8]:
sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

####<font color="#fd79a8">Heap queue <br>heap queue algorithm, also known as the priority queue algorithm<br>It makes it possible to view the data (words/scores) -  our heap, as a regular Python list<br><font color="#0abde3">heapq.nlargest(n, iterable, key=None)

In [9]:
import heapq
summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)
summary

"AloTech’in bulut ve yedeklilik altyapıları ile her koşulda hizmet verin.Kurulum maliyeti olmadan AloTech’i kullanınKurulum için ücret ödemeden, donanım yatırımı yapmadan aylar değil günler içerisinde projelerinizi canlıya alın. AloTech, verimliliği optimize edip maliyeti düşürürken en iyi müşteri deneyimini sağlamanız için ihtiyacınız olan araçları size sunan yeni nesil bir çağrı merkezi platformudur. İş uygulamalarınızı çağrı merkeziniz ile entegre ederek iş optimizasyonunda öncü olun.Önce veri güvenliği ve gizlilikAloTech'te numara maskeleme özelliği sayesinde müşterileriniz ile çağrı merkezi operasyonu çalışanlarınızın gizliliğini koruyabilirsiniz. Daha birlikte yapacağımız çok proje var!Yenilikçi ve çözüm odaklı yaklaşımlarıyla AloTech’in desteğini her zaman yanımızda hissediyoruz. Gerçekleştirdiğimiz projelerin temelinde güçlü teknoloji ve tecrübeli bir ekip yer alıyor."